In [1]:
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot as plt
from datasetsforecast.losses import rmse
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

warnings.filterwarnings("ignore")

# Helper functions
def plot(df, cv, model_name):
    plt.figure(figsize=(10, 6))
    plt.plot(df['ds'], df['y'], label='Actual', marker='o', linestyle='-', color='black')
    plt.plot(cv['ds'], cv[model_name], label=f'{model_name} Predictions', marker='x', linestyle='--')
    plt.title(f"Actual vs. {model_name} Predictions")
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

def clean_columns(feature_names):
    replace_map = {
        '[': '', ']': '',  
        '/': '_',         
        '"': '',          
        '{': '', '}': '', 
        ':': '_',         
        ',': '_',         
        ' ': '_',          
        '(': '_',         
        ')': '_',         
        '<': '',          
    }
    
    cleaned_names = []
    for name in feature_names:
        for old_char, new_char in replace_map.items():
            name = name.replace(old_char, new_char)
        name = ''.join(char if char.isalnum() else ' ' for char in name.lower())
        name = ' '.join(name.split())
        cleaned_names.append(name)
    return cleaned_names

# Load and preprocess macro data
start_date = '2009-06-01'
macro_data = pd.read_excel("macro_data.xlsx")
macro_data = macro_data.drop(0)
macro_data = macro_data.rename(columns={macro_data.columns[0]: 'ds'})
macro_data['ds'] = pd.to_datetime(macro_data['ds'], format='%Y-%m-%d')
macro_data = macro_data.fillna(method='ffill')
macro_data = macro_data[macro_data['ds'] >= start_date]
macro_data = macro_data.fillna(0)

# Load and preprocess target price data
target_data = pd.read_excel("target_rice_data.xlsx", sheet_name="Monthly Prices", header=4)
target_data = target_data.rename(columns={target_data.columns[0]: 'ds'})
target_data = target_data[['ds', 'Rice, Viet Namese 5%']]
target_data = target_data.drop([0, 1])
target_data['ds'] = target_data['ds'].str.replace('M', '-')
target_data['ds'] = target_data['ds'] + '-01'
target_data['ds'] = pd.to_datetime(target_data['ds'], format='%Y-%m-%d')
target_data['Rice, Viet Namese 5%'] = pd.to_numeric(target_data['Rice, Viet Namese 5%'], errors='coerce')
target_data = target_data[target_data['ds'] >= start_date]

# Merge data and prepare final dataframes
df = pd.merge(target_data, macro_data, on='ds', how='inner')
df = df.rename(columns={'Rice, Viet Namese 5%': 'y'})
df = df.sort_values(by='ds', ascending=True)

# Prepare feature matrix X and target vector y
X = df.drop(['ds', 'y'], axis=1)
X = X.fillna(0)
X.columns = clean_columns(X.columns)
y = df['y'].astype(float)
y = y.fillna(0)

# Create price only dataframe
price_only_df = df[['ds', 'y']].copy()
price_only_df['y'] = price_only_df['y'].astype(float)
price_only_df['unique_id'] = 'Stats_Model'
price_only_df['ds'] = price_only_df['ds'] + pd.offsets.MonthEnd(0)
price_only_df = price_only_df.fillna(method='ffill')

In [2]:
price_only_df

,ds,y,unique_id
0,2009-06-30,404.2850,Stats_Model
1,2009-07-31,410.2280,Stats_Model
2,2009-08-31,395.7175,Stats_Model
3,2009-09-30,392.8580,Stats_Model
4,2009-10-31,414.1075,Stats_Model
...,...,...,...
172,2023-10-31,616.1500,Stats_Model
173,2023-11-30,629.4100,Stats_Model
174,2023-12-31,632.9000,Stats_Model
175,2024-01-31,625.5100,Stats_Model


In [3]:
#create with_macro_df by joing price_only_df and macro_data on 'ds'. First shift 'ds' of macro_data to end of month
#drop unique_id from price_only_df
price_only_df = price_only_df.drop(columns=['unique_id'])
macro_data['ds'] = macro_data['ds'] + pd.offsets.MonthEnd(0)
#join price_only_df and macro_data on 'ds'
with_macro_df = pd.merge(price_only_df, macro_data, on='ds', how='inner')
with_macro_df.columns = clean_columns(with_macro_df.columns)
with_macro_df['unique_id'] = 'price_and_macro'
with_macro_df

,ds,y,industrial production index,ipi mining and quarrying mq,ipi manufacturing,industrial production index computer electronic and optical products,ipi electricity and gas eg,industrial production ytd coal,industrial production ytd electricity,industrial production index 1,...,no of tourist arrivals,freight carried ytd railway,freight carried ytd maritime transport,freight carried ytd road,freight carried ytd airway,no of passengers carried ytd railway,no of passengers carried ytd maritime transport,no of passengers carried ytd road,no of passengers carried ytd airway,unique_id
0,2009-06-30,404.2850,0.00,0.00,0.00,0.00,0.00,21351.800000,37600.000000,90.680349,...,0.0,3945.800,21281.500000,2.259892e+05,61.200000,5565.100,3044.600000,8.688895e+05,5239.300000,price_and_macro
1,2009-07-31,410.2280,0.00,0.00,0.00,0.00,0.00,24752.900000,44900.000000,93.312570,...,0.0,4631.700,26454.300000,2.649756e+05,70.000000,6863.700,3492.800000,1.017431e+06,6230.800000,price_and_macro
2,2009-08-31,395.7175,0.00,0.00,0.00,0.00,0.00,28689.020093,52310.000000,93.579757,...,0.0,5403.100,31556.300000,3.048197e+05,79.100000,7792.100,3934.200000,1.167531e+06,7315.100000,price_and_macro
3,2009-09-30,392.8580,0.00,0.00,0.00,0.00,0.00,31566.100000,62300.000000,94.303088,...,0.0,6081.300,33318.400000,3.449744e+05,89.800000,8597.200,4381.400000,1.322135e+06,8271.900000,price_and_macro
4,2009-10-31,414.1075,0.00,0.00,0.00,0.00,0.00,35165.500000,66500.000000,94.862443,...,0.0,6621.100,36745.900000,3.859084e+05,100.000000,9552.400,4868.200000,1.478455e+06,9104.200000,price_and_macro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2023-10-31,616.1500,162.38,77.84,175.91,251.58,200.67,40636.886564,222730.523024,95.556571,...,1468.8,3714.727,98276.466406,1.376555e+06,259.906032,5250.421,9858.183725,3.471429e+06,47401.916650,price_and_macro
173,2023-11-30,629.4100,162.42,77.13,176.36,239.48,197.90,44750.835434,244653.296420,95.556571,...,1468.8,4108.327,106611.721174,1.575268e+06,292.839442,5672.898,10471.142545,3.887923e+06,51540.354055,price_and_macro
174,2023-12-31,632.9000,162.61,72.04,177.93,231.25,192.70,48246.408316,266675.857576,95.556571,...,1468.8,4551.927,116128.106245,1.747256e+06,327.870938,6095.519,11093.694000,4.288059e+06,56032.621065,price_and_macro
175,2024-01-31,625.5100,155.22,75.31,167.08,201.14,197.93,4028.839775,22503.970018,95.556571,...,1468.8,410.300,10838.875931,1.504101e+05,36.801353,422.318,1931.176183,3.516843e+05,4148.214000,price_and_macro


# Process News dataset

In [4]:
# default library imports
import os
import tempfile
import json
from collections import Counter
from math import sqrt
from datetime import datetime

In [5]:
# Load News Data
data = pd.read_excel("news_data_loaded.xlsx")
# Normalize the 'Analysis' data
def normalize_analysis(entry):
    if isinstance(entry, str):
        try:
            # Attempt to parse the string as JSON, replacing single quotes with double quotes
            parsed = json.loads(entry.replace("\n", "").replace("'", '"'))
            # If parsed data is a dictionary, wrap it in a list; otherwise, return as-is
            return [parsed] if isinstance(parsed, dict) else parsed
        except json.JSONDecodeError:
            # Return an empty list if JSON parsing fails
            return []
    elif isinstance(entry, list):
        # Ensure all elements in the list are dictionaries, replace non-dicts with empty dicts
        return [elem if isinstance(elem, dict) else {} for elem in entry]
    # Return an empty list for any other data types
    return []

# Apply the normalization function to the 'Analysis' column
data['Analysis'] = data['Analysis'].apply(normalize_analysis)

# Drop rows with dates before 2008 and keep only Date and Analysis columns
data = data[data['Date'] >= '2008-01-01'][['Date', 'Analysis']]

data

def sum_scores(analysis):
    return sum(next(iter(item.values())) for item in analysis if isinstance(item, dict) and item)
data['Net Sentiment'] = data['Analysis'].apply(sum_scores)
data


,Date,Analysis,Net Sentiment
0,2024-03-26,"[{'global_supply': -80}, {'export_restrict': -...",-190.0
1,2024-03-20,"[{'supply': -70}, {'demand': 50}, {'weather': ...",20.0
2,2024-03-14,"[{'trade': 70}, {'import_increase': 85}, {'dro...",220.0
3,2024-03-05,"[{'trade': 80, 'export_restrict': -40, 'el_nin...",80.0
4,2024-02-28,"[{'trade': 70}, {'export_restrict': -30}, {'do...",430.0
...,...,...,...
3276,2008-01-09,"[{'rain': -80}, {'landslide': -80}, {'harvest_...",-230.0
3277,2008-01-07,"[{'production_increase': 80}, {'export_decreas...",-150.0
3278,2008-01-05,"[{'export_restrict': -60}, {'demand': 70}, {'s...",-75.0
3279,2008-01-04,"[{'trade': 50}, {'export_agreement': 100}]",150.0


In [6]:

data['Date'] = pd.to_datetime(data['Date']).sort_values()
data['Date'] = data['Date'] + pd.DateOffset(months=1)

# Set 'Date' as the index, resample to monthly frequency, and sum the values
data.set_index('Date', inplace=True)
data = data.resample('M').sum().reset_index()
data



,Date,Analysis,Net Sentiment
0,2008-02-29,"[{'trade': 50}, {'export_agreement': 100}, {'e...",1296.0
1,2008-03-31,"[{'export': 25}, {'trade': 40}, {'export_restr...",630.0
2,2008-04-30,"[{'export': 50}, {'agricultural_production': 6...",725.0
3,2008-05-31,"[{'trade': 60}, {'export_restrict': -90}, {'do...",445.0
4,2008-06-30,"[{'trade': -80}, {'export_restrict': -90}, {'l...",1203.0
...,...,...,...
190,2023-12-31,"[{'rice_price': 70}, {'export_volume': 60}, {'...",2285.0
191,2024-01-31,"[{'production': 30, 'export': 50, 'demand': 40...",40.0
192,2024-02-29,"[{'export': 90}, {'trade': 80}, {'import': 60}...",65.0
193,2024-03-31,"[{'rice_demand': 50}, {'rice_supply': -15}, {'...",670.0


In [7]:
#create sent_only_df by joining data and with_macro_df on 'ds'
sent_only_df = pd.merge(data, with_macro_df, left_on='Date', right_on='ds', how='inner').drop('ds', axis=1)
#Only keep Date, Net Sentiment, y
sent_only_df = sent_only_df[['Date', 'Net Sentiment', 'y']]
sent_only_df = sent_only_df.fillna(method='ffill')
sent_only_df['unique_id'] = 'sentiment_and_price'
#rename Date to ds
sent_only_df = sent_only_df.rename(columns={'Date': 'ds'})
sent_only_df


,ds,Net Sentiment,y,unique_id
0,2009-06-30,1065.90,404.2850,sentiment_and_price
1,2009-07-31,813.00,410.2280,sentiment_and_price
2,2009-08-31,805.00,395.7175,sentiment_and_price
3,2009-09-30,-1985.50,392.8580,sentiment_and_price
4,2009-10-31,-162.03,414.1075,sentiment_and_price
...,...,...,...,...
172,2023-10-31,-250.00,616.1500,sentiment_and_price
173,2023-11-30,1090.00,629.4100,sentiment_and_price
174,2023-12-31,2285.00,632.9000,sentiment_and_price
175,2024-01-31,40.00,625.5100,sentiment_and_price


In [8]:
# Merge data with pure_stats_df and apply forward fill
full_df= pd.merge(data, with_macro_df, left_on='Date', right_on='ds', how='inner').drop('ds', axis=1)
full_df= full_df.fillna(method='ffill')

# Rename columns and drop unnecessary ones
full_df = full_df.rename(columns={'Date': 'ds'}).drop('Analysis', axis=1)
full_df['unique_id'] = 'full_data'
#rename Net Sentiment to sentiment
full_df = full_df.rename(columns={'Net Sentiment': 'net sentiment'})
full_df 


,ds,net sentiment,y,industrial production index,ipi mining and quarrying mq,ipi manufacturing,industrial production index computer electronic and optical products,ipi electricity and gas eg,industrial production ytd coal,industrial production ytd electricity,...,no of tourist arrivals,freight carried ytd railway,freight carried ytd maritime transport,freight carried ytd road,freight carried ytd airway,no of passengers carried ytd railway,no of passengers carried ytd maritime transport,no of passengers carried ytd road,no of passengers carried ytd airway,unique_id
0,2009-06-30,1065.90,404.2850,0.00,0.00,0.00,0.00,0.00,21351.800000,37600.000000,...,0.0,3945.800,21281.500000,2.259892e+05,61.200000,5565.100,3044.600000,8.688895e+05,5239.300000,full_data
1,2009-07-31,813.00,410.2280,0.00,0.00,0.00,0.00,0.00,24752.900000,44900.000000,...,0.0,4631.700,26454.300000,2.649756e+05,70.000000,6863.700,3492.800000,1.017431e+06,6230.800000,full_data
2,2009-08-31,805.00,395.7175,0.00,0.00,0.00,0.00,0.00,28689.020093,52310.000000,...,0.0,5403.100,31556.300000,3.048197e+05,79.100000,7792.100,3934.200000,1.167531e+06,7315.100000,full_data
3,2009-09-30,-1985.50,392.8580,0.00,0.00,0.00,0.00,0.00,31566.100000,62300.000000,...,0.0,6081.300,33318.400000,3.449744e+05,89.800000,8597.200,4381.400000,1.322135e+06,8271.900000,full_data
4,2009-10-31,-162.03,414.1075,0.00,0.00,0.00,0.00,0.00,35165.500000,66500.000000,...,0.0,6621.100,36745.900000,3.859084e+05,100.000000,9552.400,4868.200000,1.478455e+06,9104.200000,full_data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2023-10-31,-250.00,616.1500,162.38,77.84,175.91,251.58,200.67,40636.886564,222730.523024,...,1468.8,3714.727,98276.466406,1.376555e+06,259.906032,5250.421,9858.183725,3.471429e+06,47401.916650,full_data
173,2023-11-30,1090.00,629.4100,162.42,77.13,176.36,239.48,197.90,44750.835434,244653.296420,...,1468.8,4108.327,106611.721174,1.575268e+06,292.839442,5672.898,10471.142545,3.887923e+06,51540.354055,full_data
174,2023-12-31,2285.00,632.9000,162.61,72.04,177.93,231.25,192.70,48246.408316,266675.857576,...,1468.8,4551.927,116128.106245,1.747256e+06,327.870938,6095.519,11093.694000,4.288059e+06,56032.621065,full_data
175,2024-01-31,40.00,625.5100,155.22,75.31,167.08,201.14,197.93,4028.839775,22503.970018,...,1468.8,410.300,10838.875931,1.504101e+05,36.801353,422.318,1931.176183,3.516843e+05,4148.214000,full_data
